In [1]:
!rm -rf /kaggle/working/multimodal-eq-sizing
!git clone -b feature/add_guards https://github.com/brianrp09232000/multimodal-eq-sizing.git /kaggle/working/multimodal-eq-sizing
!pip install -r /kaggle/working/multimodal-eq-sizing/requirements.txt

Cloning into '/kaggle/working/multimodal-eq-sizing'...
fatal: Remote branch feature/add_guards not found in upstream origin
ERROR: Could not open requirements file: [Errno 2] No such file or directory: '/kaggle/working/multimodal-eq-sizing/requirements.txt'


In [ ]:
import sys
import pathlib
import pandas as pd
import numpy as np
import kagglehub
from datetime import datetime, timedelta

In [ ]:
np.seterr(invalid="ignore")

In [ ]:
repo_root = pathlib.Path("/kaggle/working/multimodal-eq-sizing")
sys.path.append(str(repo_root))

In [ ]:
local_path = kagglehub.notebook_output_download("01_prepare_data.ipynb")
local_path

In [ ]:
from src.backtest.behavior_policy import run_behavior_policy_with_guards
from src.rl.state_builder import ensure_z_column, build_rl_dataset

In [ ]:
# 1) Load final stacked dataset from 03_calibrate_and_stack
final_path = "../input/your-final-dataset/final_dataset.csv"
df = pd.read_csv(final_path, parse_dates=["Date"])

In [ ]:
# 2) Ensure we have a z column (dummy for now)
df = ensure_z_column(df, z_col="dummy_z", seed=42)

In [ ]:
# 3) Run dumb behavior policy + guards
df_policy = run_behavior_policy_with_guards(
    df,
    z_col="dummy_z",    # later change to "z" when you have real alpha
    nav=1_000_000.0,
    allow_short=False,
    trading_enabled=True,
)

In [ ]:
# 4) Build RL training dataset (s, a, r, s')
rl_df = build_rl_dataset(
    df_policy,
    z_col="dummy_z",
    reward_return_col="excess_return",
    action_col="action_weight_raw",
    weight_col="weight_after_guards",
    extra_state_cols=["has_news", "disagreement"],  # only if those exist
    cost_bps=10.0,
    lambda_risk=0.1,
)
rl_df.head()